## Aim
#### Last experiment
In the last experiments I looked at the effect of buffer components on the signal to noise ratio (A420/A390). I narrowed down my buffer additives to Triton, BSA and Arginine, none of which seemed to interact that much. I did notice that Arginine may induce a type II shift, but I'm still including it in this experiment though because I want to be sure before I rule it out. 

#### Response
I want to test the accuracy and precision of the assay (I'll have to do some titrations as a reference point). 
I'll probably use pearsons' correlation coefficient for accuracy (once I have titration data), Stev for precision and maybe factor in a line-fitting metric. I'll have to do repeats to get a precision metric which means I'll have to do multiple plates.

#### Factors
I want to select one buffer additive from arginine, BSA and Triton. I'm testing how each of these factors interacts with the final [DMSO] and the K value. So I'll do a D-optimal design between each factor and DMSO and K. 3 sets of 3-factor experiments.



In [155]:
from PlateObjects import Block, SourcePlateCompound, AssayPlate
from EchoXMLGenerator import Generate, WellIDtoNumber

import pandas as pd
import dexpy
import dexpy.optimal
from dexpy.model import ModelOrder
from sklearn.preprocessing import MinMaxScaler
import string


def MakeDesign(additives, factors):
    # Design size is the factors plus one additive
    design = dexpy.optimal.build_optimal(len(factors)+1,order=ModelOrder.quadratic)
    cols = list(factors)
    cols.append(additives)
    design.columns = cols
    return design

def NormalizeDesign(design,factors):
    for i in design:
        # new scaler for each column
        low = factors[i][0]
        high = factors[i][1]
        scaler = MinMaxScaler(feature_range = (low, high))
        design[i] = scaler.fit_transform(design[[i]])
    design.fillna(0,inplace=True)
    return design



Factors = {'Protein Vol':[20,30],'Protein Conc':[10,20]}
Additives = {'Arginine':[100,200],'BSA':[0.1,0.5],'Triton':[0.01,0.1]}


# Just arginine today
design = pd.DataFrame([],columns=list(Factors) + list(Additives))
design = design.append(MakeDesign('Triton', Factors),sort = True)
design.dropna(inplace=True,axis=1)
Factors.update(Additives)
design = NormalizeDesign(design, Factors)
assayplate1 = AssayPlate()

for i in design.index:
    row = design.loc[i,:]
    additive = 'Triton'
    additive += ' '+str(round(row[additive],2))

    for repeat in range(2):
        block = Block(row['Protein Vol'],row['Protein Conc'], 'Arachadionic acid',additive)
        assayplate1.AddBlocks(block) 
assayplate1.MakeTransfer()
transferplan = assayplate1.TransferPlan

###design.to_csv('Triton_Dopt.csv')
design=pd.read_csv('Triton_Dopt.csv')

In [158]:
def UniqueIDs(df):
    unique = df.loc[:,['Protein Conc','Triton']].drop_duplicates()
    unique.index = list(string.ascii_uppercase[0:len(unique)])
    IDs = []
    for i in df.index:
        row = df.loc[i]
        for j in unique.index:
            if unique.loc[j,'Protein Conc'] == row.loc['Protein Conc'] and\
            unique.loc[j,'Triton'] == row.loc['Triton']:
                IDs.append(j)
    IDs = pd.Series(IDs)
    df['ID'] = IDs
    return df
design = UniqueIDs(design)
design

,Protein Conc,Protein Vol,Triton,ID
0,10.000000,20.000000,0.010000,A
1,10.000000,30.000000,0.100000,B
2,10.000000,26.363636,0.042727,C
3,10.000000,20.000000,0.100000,B
4,20.000000,20.000000,0.100000,D
5,16.363636,26.363636,0.100000,E
6,13.636364,30.000000,0.010000,F
7,20.000000,30.000000,0.067273,G
8,20.000000,22.727273,0.010000,H
9,15.454545,20.000000,0.050909,I


In [188]:
def V1(v2,C1,C2):
    return (C2*v2)/C1


def VolToAdd(RowFromExpDesign, TotalVol_ul):
    # columns in the design and keys in the stock concs need to 
    # be in the same order
    TritonConc = 10 # 
    ProteinConc = 734.262 #um

    volArg = V1(TotalVol_ul,TritonConc,RowFromExpDesign['Triton'])
    VolProt = V1(TotalVol_ul,ProteinConc,RowFromExpDesign['Protein Vol'])
    VolBuffer = TotalVol_ul- (volArg+VolProt)
    output = pd.DataFrame([[volArg,VolProt,VolBuffer]],columns = ['volTriton','VolProt','VolBuffer'])
    return output


pipettingplan = pd.DataFrame([],columns = ['volTriton','VolProt','VolBuffer'])
#pipettingplan.index=design['ID']
                             
for i in design.index:
    pipettingplan = pipettingplan.append(VolToAdd(design.loc[i,:], 10_000)) # 10 ml of each


pipettingplan.reset_index(inplace=True,drop=True)
pipettingplan = pipettingplan.round(2)


print('Total vol of prot:',pipettingplan['VolProt'].sum())
print('Total vol of Triton:',pipettingplan['volTriton'].sum())
print('Total vol of Buffer:',pipettingplan['VolBuffer'].sum())

pipettingplan['VolBuffer']-=5000
#pipettingplan = pipettingplan.sort_values('volTriton').drop_duplicates()
#(pd.Series([5000]*6,index=pipettingplan.index) - pipettingplan['VolProt']).sort_values()
(pipettingplan['VolProt']/2).sort_values()

Total vol of prot: 3342.8600000000006
Total vol of Triton: 590.91
Total vol of Buffer: 96066.22


0    136.190
3    136.190
4    136.190
9    136.190
8    154.765
2    179.525
5    179.525
1    204.285
6    204.285
7    204.285
Name: VolProt, dtype: float64

In [132]:
print('Rows:\n')
for i in design.index:
    print(i,(i*2)+1, (i*2)+2)

Rows:

0 1 2
1 3 4
2 5 6
3 7 8
4 9 10
5 11 12
6 13 14
7 15 16
8 17 18
9 19 20


In [189]:
design

,Protein Conc,Protein Vol,Triton,ID
0,10.000000,20.000000,0.010000,A
1,10.000000,30.000000,0.100000,B
2,10.000000,26.363636,0.042727,C
3,10.000000,20.000000,0.100000,B
4,20.000000,20.000000,0.100000,D
5,16.363636,26.363636,0.100000,E
6,13.636364,30.000000,0.010000,F
7,20.000000,30.000000,0.067273,G
8,20.000000,22.727273,0.010000,H
9,15.454545,20.000000,0.050909,I
